In [76]:
%matplotlib inline

import pandas as pd
import numpy as np
from scipy.stats import rv_continuous, norm, truncnorm, arcsine, chi2, gamma, triang, entropy
from scipy.special import erf
import matplotlib.pyplot as plt
from itertools import combinations_with_replacement
from time import time
from random import shuffle
from copy import deepcopy

In [41]:
gift_types = ['horse', 'ball', 'bike', 'train', 'coal', 'book', 'doll', 'blocks', 'gloves']
gift_numbers = [1000, 1100, 500, 1000, 166, 1200, 1000, 1000, 200]
gift_ids = [[item +'_'+str(i) for i in range(gift_numbers[index])] for index, item in enumerate(gift_types)]
df = pd.read_csv('best_combinations.csv')

In [42]:
def unshared_copy(inList):
    if isinstance(inList, list):
        return list( map(unshared_copy, inList) )
    return inList

def choiceful_gnome(dataframe, gift_numbers, N_packs=1000):
    dataframe = dataframe.copy().reset_index(drop=True)
    packs_left = N_packs
    items_left = gift_numbers[:]
    cart_recipes = []
    cart_scores = []
    while packs_left != 0:
        max_packs = dataframe.apply(lambda x: min([packs_left] + [(n_left//n_recipe) if n_recipe>0 else np.inf for n_left, n_recipe in zip(items_left, x[:9])]),axis=1)
        scores = dataframe['score']
        total_score = max_packs * scores
        max_index = total_score.idxmax()
        n = int(max_packs[max_index])
        if n ==0:
            print('!!!Incomplete cart!!!')
            print('Cart length = %.2d\nCart average = %.2f' % (len(cart_recipes),sum(cart_scores)/len(cart_recipes)))
            break
        max_score = scores[max_index]
        max_recipe = dataframe.iloc[max_index,:9]
        cart_recipes += [list(map(int,max_recipe))] * n
        cart_scores += [max_score] * n
        packs_left -= n
        items_left -= max_recipe * n
        if min(items_left)<0:
            print(packs_left)
            print(max_packs[max_index])
            #print(cart_recipes)
            print('packs_left = %d\n' % packs_left)
            print('n = %d' % n)
            print('max_score = %d' % max_score)
            print('max_index = %d' % max_index)
            print('max_recipe = %s\n' % list(max_recipe))
            print('items_left =\n%s' % list(items_left))
            raise Exception('Something went wrong')
    return [cart_scores, cart_recipes, list(items_left.astype(int))]
        
        
        
        

In [43]:
from collections import OrderedDict

class Recipe:
    def __init__(self, recipe=None, score=None, row=None):
        if row is not None:
            self.setrow(row)
        else:
            self.recipe = self.setr(recipe)
            self.score = score
    
    def __repr__(self):
        return 'Recipe = ' + str(self.recipe) + ', Score = ' + str(self.score)
    
    def __eq__(self, other):
        """Override the default Equals behavior"""
        if isinstance(other, self.__class__):
            return self.__dict__ == other.__dict__
        return NotImplemented

    def __ne__(self, other):
        """Define a non-equality test"""
        if isinstance(other, self.__class__):
            return not self.__eq__(other)
        return NotImplemented

    def __hash__(self):
        """Override the default hash behavior (that returns the id or the object)"""
        return hash(tuple(sorted(self.__dict__.items())))
    
    def getr(self):
        return self.recipe
    
    def gets(self):
        return self.score
    
    def setr(self, recipe):
        self.recipe = tuple([int(i) for i in recipe])
    
    def setrow(self,row):
        self.recipe = tuple(row[:9])
        self.score = row[9]
    
    
class Cart:
    cart_df = df.copy()
    cart_df = cart_df[cart_df['n_items']>=3].reset_index(drop=True)

    def __init__(self, recipe_list=None, recipe_number=None, gift_numbers=None,items_left=None, gift_types=None, max_size=1000):
        if recipe_list == None:
            recipe_list = []
        if recipe_number == None:
            recipe_number = [1] * len(recipe_list)
        if gift_numbers == None:
            gift_numbers = [1000, 1100, 500, 1000, 166, 1200, 1000, 1000, 200]
        if items_left == None:
            items_left = list(gift_numbers)
        if gift_types == None:
            gift_types = ['horse', 'ball', 'bike', 'train', 'coal', 'book', 'doll', 'blocks', 'gloves']
        if len(items_left) != len(gift_types):
            raise Excpetion('items_left and gift_types have different lengths.')
        self.items_left = list(items_left)
        self.gift_numbers = list(gift_numbers)
        self.gift_types = list(gift_types)
        self.max_size = max_size
        self.cart = dict()
        for r,n in zip(recipe_list, recipe_number):
            self.add(r,n)
    
    def __len__(self):
        return sum(self.cart.values())
    
    def __repr__(self):
        out_str = ''
        for r in self.cart:
            out_str += str(r) + ', Instances = ' + str(self.cart[r]) + '\n'
        out_str += 'Items left = ' + str(self.items_left) + '\n'
        out_str += 'Cart size = ' + str(len(self)) + '\n'
        out_str += 'Cart score = ' + str(self.getscore())
        return out_str
    
    def __gt__(self, other):
        return self.getscore() > other.getscore()
    
    def __lt__(self, other):
        return self.getscore() < other.getscore()
    
    def __eq__(self, other):
        return self.getscore() == other.getscore()
    
    def getscore(self):
        score = 0
        for r,n in self.cart.items():
            score += r.gets() * n
        return score
                   
    def set_items_left(self, items_left):
        if len(items_left) != 9:
            raise Exception('Length of items_left must be 9.')
        self.items_left = items_left
        
    
    def add(self, recipe, n=1, addmax=False):
        if addmax:
            n = int(self.maxr(recipe))
        else:
            n = int(n)
        self.items_left = [i-r*n if (i-r*n)>=0 else exec('raise Exception(\'Negative number of items left.\')') for i,r in zip(self.items_left, recipe.getr())]
        if recipe in self.cart:
            self.cart[recipe] += n
        else:
            self.cart[recipe] = n
            
    def add_max(self, recipe, n=1):
        n = int(min(n, self.maxr(recipe)))
        self.items_left = [i-r*n if (i-r*n)>=0 else exec('raise Exception(\'Negative number of items left.\')') for i,r in zip(self.items_left, recipe.getr())]
        if recipe in self.cart:
            self.cart[recipe] += n
        else:
            self.cart[recipe] = n
    
    def remove(self, recipe, n=1, removeall=False):
        if recipe not in self.cart:
            raise Exception('Item not in cart.')
        if removeall:
            n = self.cart[recipe]
        else:
            n = int(n)
        if self.cart[recipe] < n:
            raise Exception('Tried to remove too many items.')
        elif self.cart[recipe] == n:
            del self.cart[recipe]
        else:
            self.cart[recipe] -= 1
        self.items_left = [i+r*n for i,r in zip(self.items_left, recipe.getr())]
        
    def maxr(self, recipe):
        packs_left = self.max_size - len(self)
        return min([packs_left] + [(n_left//n_recipe) if n_recipe>0 else np.inf for n_left, n_recipe in zip(self.items_left, recipe.getr())])

    @staticmethod
    def _multiple_pop(li, npop):
        out = []
        npop = int(npop)
        for i in range(npop):
            out += [li.pop()]
        return out    
    
    def _recipe_list_to_ids(self, recipe, gift_ids):
            return ' '.join([' '.join(self._multiple_pop(gift_ids[i], item_number)) for i,item_number in enumerate(recipe.getr()) if item_number > 0])
  
    def get_gift_list(self, random=False):
        ids = [[item +'_'+str(i) for i in range(self.gift_numbers[index])] for index, item in enumerate(self.gift_types)]
        if random: 
            for i in ids:
                shuffle(i)
        out = []
        for r,n in self.cart.items():
            out += [self._recipe_list_to_ids(r, ids) for i in range(n)]
        return out
    
    def best_recipes(self, depth):
        dataframe = self.cart_df
        max_packs = dataframe.apply(lambda x: min([self.max_size - len(self)] + [(n_left//n_recipe) if n_recipe>0 else np.inf for n_left, n_recipe in zip(self.items_left, x[:9])]),axis=1)
        scores = dataframe['score']
        total_score = max_packs * scores
        best_scores_indices = list(total_score.sort_values(ascending=False).index[0:depth])
        out = []
        for i in best_scores_indices:
            new_recipe = Recipe(row=dataframe.iloc[i])
            out.append(new_recipe)
        return out
    
    def best_recipes_max_n(self, depth, max_n):
        dataframe = self.cart_df
        max_packs = dataframe.apply(lambda x: min([max_n] + [self.max_size - len(self)] + [(n_left//n_recipe) if n_recipe>0 else np.inf for n_left, n_recipe in zip(self.items_left, x[:9])]),axis=1)
        scores = dataframe['score']
        total_score = max_packs * scores
        best_scores_indices = list(total_score.sort_values(ascending=False).index[0:depth])
        out = []
        for i in best_scores_indices:
            new_recipe = Recipe(row=dataframe.iloc[i])            
            if max_packs.iloc[i] == 0:
                return out
            out.append(new_recipe)
        return out
    
    
    def isfull(self):
        if len(self) == self.max_size:
            return True
        elif len(self) > self.max_size:
            raise Exception('Max size exceeded.')
        else:
            return False

In [64]:
def flatten(S):
    if S == []:
        return S
    if isinstance(S[0], list):
        return flatten(S[0]) + flatten(S[1:])
    return S[:1] + flatten(S[1:])

def test(depth):
    if depth == 0:
        return [depth]
    else:
        return flatten(list(map(lambda x: test(x-1), [depth]*3)))

def normal_score(mean, var, L):
    #returns the score for a normal variable for max weight = L
    std = var**0.5
    return -(std/(np.exp((L - mean)**2/(2.*std**2))*np.sqrt(2*np.pi))) + (mean*(1 + erf((L - mean)/(np.sqrt(2)*std))))/2.

In [63]:
def mask(df, f):
    """df.mask(lambda x: x[0] < 0).mask(lambda x: x[1] > 0)"""
    return df[f(df)]

def flatten(S):
    if S == []:
        return S
    if isinstance(S[0], list):
        return flatten(S[0]) + flatten(S[1:])
    return S[:1] + flatten(S[1:])


number_of_iterations = 0
finished_leaves = 0

def searchful_gnome(cart, depth, max_n):
    global number_of_iterations
    global finished_leaves
    
    if cart.isfull():
        #####
        print('Number of finished leaves: %d' % finished_leaves)
        finished_leaves+=1
        #####
        return [cart]  
    
    best_recipes = cart.best_recipes_max_n(depth, max_n)
    
    
    if best_recipes == []:
        print('!!!Cart is not full!!!')
        print('Number of finished leaves: %d' % finished_leaves)
        finished_leaves+=1        
        return cart
    
    new_carts = []
    for br in best_recipes:
        nc = deepcopy(cart)
        nc.add_max(br, max_n)
        new_carts.append(nc)
    
    out = []
    #####
    print('Iteration number: %d' % number_of_iterations)
    number_of_iterations+=1
    #####
    return flatten(list(map(searchful_gnome, new_carts, [depth]*depth, [max_n]*depth)))


def lazy_searchful_gnome(cart, depth, max_n):
    global number_of_iterations
    global finished_leaves
    
    if cart.isfull():
        #####
        print('Number of finished leaves: %d' % finished_leaves)
        finished_leaves+=1
        #####
        return [cart]  
    
    best_recipes = cart.best_recipes_max_n(depth, max_n)
    
    
    if best_recipes == []:
        print('!!!Cart is not full!!!')
        print('Number of finished leaves: %d' % finished_leaves)
        finished_leaves+=1        
        return cart
    
    new_carts = []
    for br in best_recipes:
        nc = deepcopy(cart)
        nc.add_max(br, max_n)
        new_carts.append(nc)
    
    out = []
    #####
    print('Iteration number: %d' % number_of_iterations)
    number_of_iterations+=1
    #####
    depth = depth-1
    return flatten(list(map(searchful_gnome, new_carts, [depth]*depth, [max_n]*depth)))
    

In [67]:
gift_means = [5.0352756509738335, 2.0, 20.551127030413816, 10.276239313394925, 23.5, 2.0, 5.0, 11.67, 1.4]
gift_vars = [3.8223773735792048, 0.09, 88.59, 22.16, 276.125, 4.0, 5.0, 9.722, 1.973]

In [69]:
gift_types

['horse', 'ball', 'bike', 'train', 'coal', 'book', 'doll', 'blocks', 'gloves']

In [70]:
bike_stats = [20.551127030413816, 88.59]
coal_stats = [23.5, 276.125]

In [72]:
bike_3 = [3*i for i in bike_stats]

In [79]:
normal_score(2*bike_stats[0], 2*bike_stats[1], 50)

26.500653713995405

In [83]:
df[df['bike']>=1]

,horse,ball,bike,train,coal,book,doll,blocks,gloves,score,n_items,max_packs
15351,0,9,1,0,0,0,0,0,0,32.383536,10,122
15391,0,8,1,0,0,0,0,0,0,32.354085,9,137
15617,1,6,1,0,0,0,0,0,0,32.188132,8,183
15640,0,7,1,0,0,0,0,0,2,32.170186,10,100
15703,0,7,1,0,0,0,0,0,1,32.121049,9,157
15723,0,6,1,0,0,0,1,0,0,32.106926,8,183
15735,0,7,1,0,0,1,0,0,0,32.100477,9,157
15825,1,5,1,0,0,0,0,0,1,32.039996,8,200
15885,0,6,1,0,0,0,0,0,3,32.005885,10,66
15968,0,5,1,0,0,0,1,0,1,31.960203,8,200


In [62]:
then = time()
N = 10
for i in range(N):
    max_packs = b.apply(lambda x: min([(n_left//n_recipe) if n_recipe>0 else np.inf for n_left, n_recipe in zip(gift_numbers, x[:9])]),axis=1)
print('Average execution time: %.2fs' % ((time()-then)/N))

Average execution time: 2.57s
